# Adaptive Bond Dimension: CPU↔GPU Handoff

This notebook demonstrates how Maestro makes it trivial to switch between CPU and GPU backends during MPS time evolution.

**The key insight:**
- At **low bond dimension** ($\chi$), CPU is faster — no GPU transfer overhead
- At **high $\chi$**, GPU wins — tensor contractions benefit from parallelism
- **Entanglement grows** during time evolution → $\chi$ must increase
- Maestro lets you switch with a **single argument change**

## Setup

In [ ]:
import numpy as np
import time
import maestro
from maestro.circuits import QuantumCircuit
import matplotlib.pyplot as plt

## Step 1: The Physical System

We use the transverse-field Ising model (TFIM) on a 2D lattice:

$$H = -J \sum_{\langle i,j \rangle} Z_i Z_j - h \sum_i X_i$$

Time evolution via Trotterization generates increasing entanglement, which requires higher $\chi$ for accurate MPS simulation.

In [ ]:
# System parameters
LX, LY = 6, 6
N_QUBITS = LX * LY
J, H_FIELD = 1.0, 1.0
DT = 0.2
N_STEPS = 10

# Bond dimension settings
CHI_LOW = 16    # Fast, approximate
CHI_HIGH = 64   # Accurate, expensive

# Set to True if you have an NVIDIA GPU with cuQuantum
USE_GPU = False

print(f"Lattice: {LX}×{LY} = {N_QUBITS} qubits")
print(f"Time: T={N_STEPS*DT:.1f}, {N_STEPS} steps")
print(f"Bond dimensions: χ_low={CHI_LOW}, χ_high={CHI_HIGH}")
print(f"GPU: {'Available' if USE_GPU else 'CPU only'}")

In [ ]:
def get_nn_bonds(lx, ly):
    """Nearest-neighbor bonds on a 2D square lattice."""
    bonds = []
    for x in range(lx):
        for y in range(ly):
            q = x * ly + y
            if x + 1 < lx:
                bonds.append((q, (x + 1) * ly + y))
            if y + 1 < ly:
                bonds.append((q, q + 1))
    return bonds

def build_pauli_observable(n_qubits, pauli_map):
    """Build a Pauli observable string."""
    labels = ['I'] * n_qubits
    for qubit, pauli in pauli_map.items():
        labels[qubit] = pauli
    return ''.join(labels)

def build_tfim_circuit(n, bonds, j, h, dt, n_steps):
    """Build a Trotterized TFIM circuit."""
    qc = QuantumCircuit()
    for q in range(n):
        qc.h(q)
    for _ in range(n_steps):
        for q1, q2 in bonds:
            qc.cx(q1, q2)
            qc.rz(q2, 2.0 * j * dt)
            qc.cx(q1, q2)
        for q in range(n):
            qc.h(q)
            qc.rz(q, 2.0 * h * dt)
            qc.h(q)
    return qc

bonds = get_nn_bonds(LX, LY)
print(f"Bonds: {len(bonds)}")

In [ ]:
def compute_energy(qc, n, bonds, j, h, chi, use_gpu=False):
    """Compute TFIM energy via MPS estimate()."""
    obs = []
    for q1, q2 in bonds:
        obs.append(build_pauli_observable(n, {q1: 'Z', q2: 'Z'}))
    for q in range(n):
        obs.append(build_pauli_observable(n, {q: 'X'}))
    
    sim_type = (maestro.SimulatorType.CuQuantum if use_gpu
                else maestro.SimulatorType.QCSim)
    
    result = qc.estimate(
        simulator_type=sim_type,
        simulation_type=maestro.SimulationType.MatrixProductState,
        observables=obs,
        max_bond_dimension=chi,
    )
    
    exp_vals = result['expectation_values']
    n_bonds = len(bonds)
    e_zz = sum(-j * exp_vals[i] for i in range(n_bonds))
    e_x = sum(-h * exp_vals[n_bonds + i] for i in range(n))
    return e_zz + e_x

print("✓ Energy computation function defined")

## Step 2: Compare Low χ vs High χ

First, let's see the difference in accuracy and speed between low and high bond dimensions.

In [ ]:
def run_time_evolution(chi, use_gpu=False, label=""):
    """Run full time evolution at a fixed χ."""
    print(f"\n── {label}: χ={chi}, {'GPU' if use_gpu else 'CPU'} ──")
    
    energies, times, step_times = [], [], []
    
    for step in range(N_STEPS + 1):
        t0 = time.time()
        if step == 0:
            energy = -N_QUBITS
        else:
            qc = build_tfim_circuit(N_QUBITS, bonds, J, H_FIELD, DT, step)
            energy = compute_energy(qc, N_QUBITS, bonds, J, H_FIELD, chi, use_gpu)
        wall = time.time() - t0
        
        energies.append(energy)
        times.append(step * DT)
        step_times.append(wall)
        
        if step % 2 == 0:
            print(f"  step {step:2d}  t={step*DT:.2f}  E={energy:10.4f}  ({wall:.3f}s)")
    
    avg = np.mean(step_times[1:])
    print(f"  → avg: {avg:.3f}s/step, total: {sum(step_times):.1f}s")
    
    return {'energies': energies, 'times': times, 
            'step_times': step_times, 'avg': avg,
            'chi': chi, 'use_gpu': use_gpu, 'label': label}

In [ ]:
# Run 1: Low χ on CPU
r_low = run_time_evolution(CHI_LOW, use_gpu=False, label="Low χ (CPU)")

In [ ]:
# Run 2: High χ on CPU
r_high = run_time_evolution(CHI_HIGH, use_gpu=False, label="High χ (CPU)")

In [ ]:
# Run 3: High χ on GPU (if available)
r_gpu = None
if USE_GPU:
    r_gpu = run_time_evolution(CHI_HIGH, use_gpu=True, label="High χ (GPU)")
else:
    print("Skipping GPU run (set USE_GPU = True to enable)")

## Step 3: Adaptive Handoff

Start at low χ (fast, CPU). When the energy change signals growing entanglement, switch to high χ (GPU if available).

Notice how **the code is identical** — we just change `max_bond_dimension` and `simulator_type`.

In [ ]:
THRESHOLD = 0.5  # Handoff when |ΔE| exceeds this

print(f"\n── ADAPTIVE: χ={CHI_LOW} → χ={CHI_HIGH} ──")
print(f"Handoff when |ΔE| > {THRESHOLD}\n")

a_energies, a_times, a_step_times, a_backends = [], [], [], []
switched = False

for step in range(N_STEPS + 1):
    chi = CHI_LOW if not switched else CHI_HIGH
    use_gpu = False if not switched else USE_GPU
    backend = 'low' if not switched else 'high'
    
    t0 = time.time()
    if step == 0:
        energy = -N_QUBITS
    else:
        qc = build_tfim_circuit(N_QUBITS, bonds, J, H_FIELD, DT, step)
        energy = compute_energy(qc, N_QUBITS, bonds, J, H_FIELD, chi, use_gpu)
    wall = time.time() - t0
    
    a_energies.append(energy)
    a_times.append(step * DT)
    a_step_times.append(wall)
    a_backends.append(backend)
    
    label = f"CPU χ={CHI_LOW}" if not switched else f"{'GPU' if USE_GPU else 'CPU'} χ={CHI_HIGH}"
    if step % 2 == 0:
        print(f"  step {step:2d}  t={step*DT:.2f}  E={energy:10.4f}  {label}  ({wall:.3f}s)")
    
    if not switched and step >= 2:
        if abs(a_energies[-1] - a_energies[-2]) > THRESHOLD:
            switched = True
            print(f"\n  ⚡ HANDOFF at step {step}: switching to {label}\n")

print(f"\n✓ Adaptive run complete")
print(f"  Low-χ steps: {sum(1 for b in a_backends if b == 'low')}")
print(f"  High-χ steps: {sum(1 for b in a_backends if b == 'high')}")

## Step 4: Visualization

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# Left: Energy evolution
ax1 = axes[0]
ax1.plot(r_low['times'], r_low['energies'], '--',
         color='#2196F3', linewidth=1.5, alpha=0.6,
         label=f'CPU χ={CHI_LOW}')
ax1.plot(r_high['times'], r_high['energies'], '--',
         color='#FF9800', linewidth=1.5, alpha=0.6,
         label=f'CPU χ={CHI_HIGH}')
if r_gpu:
    ax1.plot(r_gpu['times'], r_gpu['energies'], '--',
             color='#4CAF50', linewidth=1.5, alpha=0.6,
             label=f'GPU χ={CHI_HIGH}')

# Adaptive: color by backend
for i in range(1, len(a_times)):
    c = '#2196F3' if a_backends[i] == 'low' else '#E91E63'
    ax1.plot(a_times[i-1:i+1], a_energies[i-1:i+1], '-',
             color=c, linewidth=3)
ax1.plot([], [], '-', color='#2196F3', linewidth=3,
         label=f'Adaptive low (χ={CHI_LOW})')
ax1.plot([], [], '-', color='#E91E63', linewidth=3,
         label=f'Adaptive high (χ={CHI_HIGH})')

ax1.set_xlabel('Simulation Time t', fontsize=12)
ax1.set_ylabel('Energy E(t)', fontsize=12)
ax1.set_title(f'Time Evolution — {LX}×{LY} TFIM', fontsize=14)
ax1.legend(fontsize=9)
ax1.grid(alpha=0.3)

# Right: Per-step timing
ax2 = axes[1]
bar_labels = [f'CPU\nχ={CHI_LOW}', f'CPU\nχ={CHI_HIGH}']
bar_vals = [r_low['avg'], r_high['avg']]
bar_colors = ['#2196F3', '#FF9800']

if r_gpu:
    bar_labels.append(f'GPU\nχ={CHI_HIGH}')
    bar_vals.append(r_gpu['avg'])
    bar_colors.append('#4CAF50')

bar_labels.append(f'Adaptive\n{CHI_LOW}→{CHI_HIGH}')
bar_vals.append(np.mean(a_step_times[1:]))
bar_colors.append('#9C27B0')

bars = ax2.bar(range(len(bar_vals)), bar_vals,
               color=bar_colors, edgecolor='black', alpha=0.8)
ax2.set_xticks(range(len(bar_vals)))
ax2.set_xticklabels(bar_labels, fontsize=10)
ax2.set_ylabel('Avg Time per Step (s)', fontsize=12)
ax2.set_title('Per-Step Cost', fontsize=14)
ax2.grid(alpha=0.3, axis='y')

for bar, val in zip(bars, bar_vals):
    ax2.annotate(f'{val:.3f}s', (bar.get_x() + bar.get_width()/2, val),
                 textcoords='offset points', xytext=(0, 5),
                 ha='center', fontsize=9, fontweight='bold')

plt.tight_layout()
plt.show()

## The Key Takeaway: Switching Backends is One Line

The entire CPU→GPU switch is just changing the `simulator_type` argument:

In [ ]:
# CPU backend
# result = qc.estimate(
#     simulator_type=maestro.SimulatorType.QCSim,       # ← CPU
#     simulation_type=maestro.SimulationType.MatrixProductState,
#     max_bond_dimension=64,
# )

# GPU backend — same code, one argument changed
# result = qc.estimate(
#     simulator_type=maestro.SimulatorType.CuQuantum,   # ← GPU
#     simulation_type=maestro.SimulationType.MatrixProductState,
#     max_bond_dimension=64,
# )

print("Same API. Same code. Just change simulator_type.")
print("No code rewrite. No separate GPU code paths.")
print("Maestro handles the backend switch transparently.")

## Summary

**What we demonstrated:**
- Low $\chi$ is fast but loses accuracy as entanglement grows
- High $\chi$ is accurate but ~20× slower per step
- GPU acceleration makes high $\chi$ practical (when available)
- **Adaptive switching** gives you accuracy when it matters, speed when it doesn't
- Switching between CPU/GPU backends is a **single argument change**

**Key Maestro APIs:**
- `SimulatorType.QCSim` — CPU backend
- `SimulatorType.CuQuantum` — GPU backend
- `max_bond_dimension=χ` — accuracy vs speed control
- `qc.estimate(observables=...)` — compute expectation values

**To try with GPU:** Set `USE_GPU = True` in Step 1 (requires NVIDIA GPU + cuQuantum).